In [20]:
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

sectors_physics_and_astronomy={
'Education':'https://www.glassdoor.com/Job/netherlands-social-science-research-assistant-jobs-SRCH_IL.0,11_IN178_KO12,45.htm?industryNId=10009', 
"Financial-Services":'https://www.glassdoor.com/Job/netherlands-social-science-research-assistant-jobs-SRCH_IL.0,11_IN178_KO12,45.htm?industryNId=10010',
"Healthcare":'https://www.glassdoor.com/Job/netherlands-social-science-research-assistant-jobs-SRCH_IL.0,11_IN178_KO12,45.htm?industryNId=10012', 
"Manufacturing":'https://www.glassdoor.com/Job/netherlands-social-science-research-assistant-jobs-SRCH_IL.0,11_IN178_KO12,45.htm?industryNId=10015',  
} 



for j in range(0, len(sectors_physics_and_astronomy.keys())):
    options = Options()
    a='user-data-dir=C:/Users/dell/AppData/Local/Google/Chrome/User Data/Profile 17'
    options.add_argument(a)
    # PATH = "/Users/yourPath/Desktop/chromedriver"
    driver = webdriver.Chrome(options=options)
    site = list(sectors_physics_and_astronomy.values())[j]
    job_desc_Math_teacher={}
    driver.get(site)
    time.sleep(20)
    initialScroll = 0
    finalScroll = 1000
    start = time.time()
    while True:
        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        initialScroll = finalScroll
        finalScroll += 1000
        time.sleep(3)
        end = time.time()
        if round(end - start) > 25:
            break
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    
    import re
    jobs=soup.find_all('li',{'class':re.compile("^react-job-listing")})
      
    print(jobs)
    images=[]
    salaries=[]
    companies=[]
    job_titles=[]
    locations=[]

    for pro in jobs:
        try:
            image=pro.find('img')['src']
            images.append(image)
        except Exception as e:
            print(f"images_error--> {e}")
            images.append("nan")
        try:
            salary=pro.find('span', {"data-test":"detailSalary"}).text.strip()
            salaries.append(salary)
        except Exception as e:
            print(f"salaries_error--> {e}")
            salaries.append("nan")
        try:
            company=pro.find('a', {"class":"css-l2wjgv e1n63ojh0 jobLink"}).text.strip()
            companies.append(company)
        except Exception as e:
            print(f"companies_error--> {e}")
            companies.append("nan")
        try:
            job=pro.find('a', {"class":"jobLink css-1rd3saf eigr9kq2"}).text.strip()
            job_titles.append(job)
        except Exception as e:
            print(f"job_titles_error--> {e}")
            job_titles.append("nan")
        try:
            loc=pro.find('span', {"data-test":"emp-location"}).text.strip()
            locations.append(loc)
        except Exception as e:
            print(f"locations_error--> {e}")
            locations.append("nan")

    try:                                
        for i in range(1, len(jobs)+1):
            job_desc_butt=driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div[2]/section/article/div[1]/ul/li['+str(i)+']')
            job_desc_butt.click()
            time.sleep(7)
            src = driver.page_source
            soup = BeautifulSoup(src, 'lxml')
            job_descript=soup.find('div', {'class':'jobDescriptionContent desc'})
            job_description=job_descript.get_text()
            job_desc_Math_teacher[i]=job_description
    except Exception as e:
        print(e)

    pros=[]
    cons=[]

    driver.close()
    driver = webdriver.Chrome(options=options)
    site1 = "https://www.glassdoor.com/Search/results.htm?keyword=&locId=178&locT=N&locName=Netherlands"

    for comp in companies:   
        driver.get(site1)
        time.sleep(20)                         
        comp_input=driver.find_element(By.XPATH, '//*[@id="sc.keyword"] ')  
        time.sleep(5)
        comp_input.click()
        comp_input.clear()
        comp_input.send_keys(comp)
        time.sleep(5)
        try:
            search_button=driver.find_element(By.XPATH, '//*[@id="scBar"]/div/button')
            search_button.click()
            time.sleep(5)

            first_comp=driver.find_element(By.XPATH, '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]/a[1]')
            first_comp.click()
            time.sleep(5)
                                                                #eiCell cell reviews 
            comp_review_button=driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/div/div/div[1]/article[1]/div[1]/div[2]/div/div[2]/div/div[1]/div/a[1]')
            comp_review_button.click()
            time.sleep(5)
            
            initialScroll = 0                    
            finalScroll = 1000
            start = time.time()
            while True:
                driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
                initialScroll = finalScroll
                finalScroll += 1000
                time.sleep(3)
                end = time.time()
                if round(end - start) > 20:
                    break
            src = driver.page_source
            soup = BeautifulSoup(src, 'lxml')

            a =soup.find_all('p', {'class':"mt-0 mb-0 pb v2_EIReviewDetailsV2bodyColor v2EIReviewDetailsV2lineHeightLarge v2EIReviewDetailsV2_isCollapsed"})
            
            list_cons=[]
            list_pros=[]
            for i in a:
                try:
                    list_cons.append(i.find('span', {'data-test':"cons"}).text.strip())
                except Exception as e:
                    print(e)
            for i in a:
                try:
                    list_pros.append(i.find('span', {'data-test':"pros"}).text.strip())
                except Exception as e:
                    print(e)
            if len(list_cons)==0:
                list_cons.append('NG')
            if len(list_pros)==0:
                list_pros.append('NG')

            pros.append(list_pros)
            cons.append(list_cons)
        except Exception as e:
            print(e)
            pros.append("NG")
            cons.append("NG")

    driver.close()

    import pandas as pd
    print("creating df")
    df_teache=pd.DataFrame({
        "job_titles":job_titles,
        "job_description":list(job_desc_Math_teacher.values()),
        "images":images,
        "locations":locations,
        "salaries":salaries,
        "companies":companies,
        "pros":pros,
        "cons":cons
        }
    )
    df_teache.to_csv(f"datas/datas_with_study/Communication-Social-Science_{list(sectors_physics_and_astronomy.keys())[j]}_last.csv", sep=',', index=False)

[<li class="react-job-listing css-1tmvqu eigr9kq3" data-ad-order-id="1136043" data-adv-type="GENERAL" data-brandviews="BRAND:n=jsearch-job-listing:eid=651704:jlid=1008538759100" data-id="1008538759100" data-is-easy-apply="false" data-is-organic-job="true" data-job-loc="Nijmegen" data-job-loc-id="3034454" data-job-loc-type="C" data-normalize-job-title="Research Assistant" data-selected="true" data-sgoc-id="1019" data-test="jobListing" data-triggered-brandview="" style=""><div class="d-flex flex-column css-x75kgh e1rrn5ka3"><a class="jobLink" href="/partner/jobListing.htm?pos=101&amp;ao=1136043&amp;s=58&amp;guid=0000018718dd7261997ba19b6dbaa4f6&amp;src=GD_JOB_AD&amp;t=SR&amp;vt=w&amp;uido=1FF7FE9E3393247AE0F74A594CA82F64&amp;cs=1_dcca0708&amp;cb=1679749378877&amp;jobListingId=1008538759100&amp;jrtk=3-0-1gscdqsk2kuic801-1gscdqsle2bg1000-b827f560253afaab-" rel="nofollow noopener noreferrer" target="_blank" title="Radboud Universiteit"><span class="css-9ujsbx euyrj9o1" data-test="emp-logo">

In [7]:
df_teache.locations

0    The Hague
1    The Hague
2    Amsterdam
3    Amsterdam
4    The Hague
5    The Hague
Name: locations, dtype: object

Done
Actuariële Wetenschappen={
'finanicial-services':'https://www.glassdoor.com/Job/netherlands--actuari%C3%ABle-wetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,37.htm?industryNId=10010', 
"Insurance":'https://www.glassdoor.com/Job/netherlands--actuari%C3%ABle-wetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,37.htm?industryNId=10014', 
"Information-Technology":'https://www.glassdoor.com/Job/netherlands--actuari%C3%ABle-wetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,37.htm?industryNId=10013'
}

Done
Ancient studies={
'finanicial-services':'https://www.glassdoor.com/Job/netherlands-studies-jobs-SRCH_IL.0,11_IN178_KO12,19.htm?industryNId=10010', 
"Education":'https://www.glassdoor.com/Job/netherlands-studies-jobs-SRCH_IL.0,11_IN178_KO12,19.htm?industryNId=10009', 
"Information-Technology":'https://www.glassdoor.com/Job/netherlands-studies-jobs-SRCH_IL.0,11_IN178_KO12,19.htm?industryNId=10013'
}

done 
 Business administration (bedrijfskunde)={
'finanicial-services':'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10010', 
"Pharmaceutical & Biotechnology":'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10005', 
"Information-Technology":'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10013'
Hotels-Travel-Accommodation':'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10025', 
"Management-Consulting":'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10006', 
}

rj
Sales Manager. ...
Business Consultant. ...
Financial Analyst. ...
Market Research Analyst. ...
Human Resources (HR) Specialist. ...
Loan Officer. ...
Meeting, Convention and Event Planner. ...
Business Consultant.

Done
business analytics={
'Education':'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10009', 
"Media-Communication":'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10016',
"Financial Services":'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10010', 
"Insurance":'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10014', 
"Energy-Mining-Utilities":'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10019',  
}

rj
Business Analytics Consultant
Senior Business Intelligence Analyst
Senior Business Analyst
Data Analysis 
Senior Data Analytics

done
Cognition, Language and Communication={
'Energy-Mining & Utilities':'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10019', 
"Financial-services":'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10010',
"Hotels-Travel-Accommodation":'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10025', 
"Pharmaceutical-Biotechnology":'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10005', 
"Human-Resources-Staffing":'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10026',  
}

rj

university instructor,
human factors consultant, 
industrial-organizational manager,
communication trainer,
communication consultant,
(online) content manager,
editor,
copywriter,
journalist
Business and marketing
Information technology
Health care
Law
Education
Research and development
Government and politics
Psychology

done

 Communicatiewetenschap={
'Education':'https://www.glassdoor.com/Job/netherlands--communicatiewetenschap-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?radius=0&industryNId=10009', 
"Human-Resources-Staffing":'https://www.glassdoor.com/Job/netherlands--communicatiewetenschap-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?radius=0&industryNId=10026',
"Media-Communication":'https://www.glassdoor.com/Job/netherlands--communicatiewetenschap-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?radius=0&industryNId=10016',  
}

rj

Public Relations Specialist
Marketing Manager
Social Media Manager
Advertising Account Executive
Corporate Communications Manager
Market Research Analyst
Media Planner
Event Planner
Content Strategist
Journalist